In [39]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [40]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [41]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "3328138531413500389", 2, "feature")

In [42]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [43]:
y = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "12590084917259381164", 2, "feature")

## Vecchia funzione

In [64]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = box(listOfModels[1])
			for theModel in listOfModels[2:end]
				modelMin,modelMax = box(theModel)
				for (k,val) in enumerate(modelMin)
					if val < theMin[k]
						theMin[k]=val
					end
				end
				for (k,val) in enumerate(modelMax)
					if val > theMax[k]
						theMax[k]=val
					end
				end
			end
		end
		return [theMin,theMax]

	elseif (isa(model,Tuple) ||isa(model,Array))&& (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end

	return [theMin,theMax]
end

box (generic function with 1 method)

In [65]:
@btime box(x)

  6.200 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [66]:
@btime box(y)

  15.899 μs (118 allocations: 5.55 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

## Nuove funzioni

In [74]:
function evalBox(listOfModels)
	theMin,theMax = box(listOfModels[1])
	for theModel in listOfModels[2:end]
		modelMin,modelMax= box(theModel)
		for (k,val) in enumerate(modelMin)
			if val < theMin[k]
				theMin[k]=val
			end
		end
		for (k,val) in enumerate(modelMax)
			if val > theMax[k]
				theMax[k]=val
			end
		end
	end
	return theMin,theMax
end

evalBox (generic function with 1 method)

In [77]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = evalBox(listOfModels)
		end
        
		return [theMin,theMax]

	elseif (isa(model,Tuple) || isa(model,Array)) && (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end
    
	return [theMin,theMax]
end

box (generic function with 1 method)

In [78]:
@btime box(x)

  6.160 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [79]:
@btime box(y)

  15.500 μs (116 allocations: 5.48 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]